In [1]:
#this is a version 2 using t2-small language model to generate the questions

In [2]:
#!pip install torch
#!pip install --upgrade typing_extensions torch
#!pip install sentencepiece

In [3]:
import tensorflow as tf
import numpy as np
import random
import json

2025-03-05 07:14:15.813033: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 07:14:15.825727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741158855.837642   91786 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741158855.841171   91786 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 07:14:15.855816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
import os
current_directory = os.getcwd()
print(current_directory)

/home/ladans/Reinforcement learning/GPT


In [5]:
# Load JSON file
with open("keywords.json", "r") as f:
    data = json.load(f)

# Convert JSON list into a dictionary
keywords = {entry["category"]: [item["keyword"] for item in entry["words"]] for entry in data}

# Print to verify structure
print("Processed Keywords:", keywords)


Processed Keywords: {'country': ['afghanistan', 'albania', 'algeria', 'andorra', 'angola', 'antigua and barbuda', 'argentina', 'armenia', 'australia', 'austria', 'azerbaijan', 'bahrain', 'bangladesh', 'barbados', 'belarus', 'belgium', 'belize', 'benin', 'bhutan', 'bolivia', 'bosnia and herzegovina', 'botswana', 'brazil', 'brunei', 'bulgaria', 'burkina faso', 'burundi', 'cambodia', 'cameroon', 'canada', 'cape verde', 'central african republic', 'chad', 'chile', 'china', 'colombia', 'comoros', 'congo', 'costa rica', 'croatia', 'cuba', 'cyprus', 'czech republic', 'democratic republic of the congo', 'denmark', 'djibouti', 'dominica', 'dominican republic', 'ecuador', 'egypt', 'el salvador', 'england', 'equatorial guinea', 'eritrea', 'estonia', 'ethiopia', 'federated states of micronesia', 'finland', 'france', 'gabon', 'gambia', 'georgia', 'germany', 'ghana', 'greece', 'grenada', 'guatemala', 'guinea', 'guinea bissau', 'guyana', 'haiti', 'honduras', 'hungary', 'iceland', 'india', 'indonesia'

In [6]:
# Load JSON data
with open("keywords.json", "r") as f:
    data = json.load(f)

# Convert JSON to a structured dictionary
keywords = {}
for entry in data:
    category = entry["category"]
    words = [item["keyword"] for item in entry["words"]]
    keywords[category] = words

# Select a random category
category = random.choice(list(keywords.keys()))

# Select a secret word from that category
secret_word = random.choice(keywords[category])

print(f"Category: {category} (Secret Word: {secret_word})")

# Initialize game state
state = f"You are playing 20 Questions. Think of a {category}."
asked_questions = []

Category: city (Secret Word: geneva switzerland)


In [7]:
import gym
import random
import numpy as np
from gym import spaces
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

from transformers import T5ForConditionalGeneration, T5Tokenizer

class T5QuestionGenerator:
    def __init__(self, model_name="t5-small"):
        self.tokenizer = T5Tokenizer.from_pretrained(model_name)
        self.model = T5ForConditionalGeneration.from_pretrained(model_name)

    def generate_question(self, category, keywords):
        """Generates a question using T5-small based on category and keywords."""
        truncated_keywords = ", ".join(keywords[:5])  # Limit to 5 keywords for efficiency
        input_text = f"Generate a question about {category}: {truncated_keywords}"

        inputs = self.tokenizer(input_text, return_tensors="pt", truncation=True, padding="max_length", max_length=50)

        # Generate question
        outputs = self.model.generate(inputs["input_ids"], max_new_tokens=20, num_return_sequences=1, do_sample=True, temperature=0.7)

        question = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return question

In [8]:
import gym
import random
import numpy as np
from gym import spaces

class TwentyQuestionsEnv(gym.Env):
    def __init__(self, keywords):
        super(TwentyQuestionsEnv, self).__init__()

        # Initialize T5 question generator
        self.qg = T5QuestionGenerator()
        self.keywords = keywords

        # Select a random category and word
        self.category = random.choice(list(self.keywords.keys()))
        self.secret_word = random.choice(self.keywords[self.category])

        # Define action & observation space
        self.action_space = spaces.Discrete(20)  # Max 20 questions
        self.observation_space = spaces.Box(low=0, high=1, shape=(20,), dtype=np.float32)

        # Initialize game state
        self.asked_questions = []
        self.done = False
        self.steps = 0
        self.max_steps = 20

    def step(self, action):
        """Agent asks a question and receives a response"""
        if self.done:
            return np.array(self._get_state()), 0, True, {}

        # Generate a question using T5-small
        question = self.qg.generate_question(self.category, self.keywords[self.category])

        # Simulate the answer
        answer = self._simulate_answer(question)

        # Calculate reward
        reward = self._calculate_reward(question, answer)

        # Track asked questions
        self.asked_questions.append((question, answer))

        # Check termination condition
        self.steps += 1
        if self.steps >= self.max_steps or answer == "Yes, correct!":
            self.done = True

        return np.array(self._get_state()), reward, self.done, {}

    def reset(self):
        """Resets the environment for a new game"""
        self.category = random.choice(list(self.keywords.keys()))
        self.secret_word = random.choice(self.keywords[self.category])
        self.asked_questions = []
        self.done = False
        self.steps = 0
        return np.array(self._get_state())

    def _get_state(self):
        """Returns the current game state"""
        return [len(self.asked_questions)]  # Simplified representation

    def _simulate_answer(self, question):
        """Simulates Yes/No/Maybe answers"""
        if any(keyword in question.lower() for keyword in self.keywords[self.category]):
            return random.choices(["Yes", "Maybe"], weights=[0.8, 0.2], k=1)[0]
        return random.choices(["No", "Maybe"], weights=[0.7, 0.3], k=1)[0]

    def _calculate_reward(self, question, answer):
        """Rewards/Penalizes the agent based on question quality"""
        if answer == "Yes":
            return 1  # Good question
        elif answer == "Maybe":
            return -0.5  # Weak question
        elif answer == "No":
            return -2  # Bad question
        return -1  # Default penalty


In [9]:
import tensorflow as tf

import tensorflow as tf
import numpy as np

class PPOAgent:
    def __init__(self, state_size, action_size, lr=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.lr = lr

        # Create models
        self.policy_model = self._build_policy_model()
        self.value_model = self._build_value_model()

        # 🔹 Ensure models are built before use
        self.policy_model.build((None, self.state_size))
        self.value_model.build((None, self.state_size))

        # Optimizers
        self.policy_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr)
        self.value_optimizer = tf.keras.optimizers.Adam(learning_rate=self.lr)

    def _build_policy_model(self):
        """Defines the policy network (actor)."""
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', input_shape=(self.state_size,)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(self.action_size, activation='softmax')
        ])
        return model

    def _build_value_model(self):
        """Defines the value network (critic)."""
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation='relu', input_shape=(self.state_size,)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(1, activation='linear')
        ])
        return model

    def get_action(self, state):
        """Selects an action (question)"""
        state = np.array([state], dtype=np.float32)  # 🔹 Ensure correct shape
        probs = self.policy_model.predict(state, verbose=0)[0]
        return np.random.choice(self.action_size, p=probs)

    def train(self, states, actions, rewards):
        """Trains PPO Agent"""
        states = np.array(states, dtype=np.float32)
        actions = np.array(actions, dtype=np.int32)
        rewards = np.array(rewards, dtype=np.float32)

        # Compute advantages
        advantages = rewards - self.value_model.predict(states, verbose=0).flatten()

        with tf.GradientTape() as policy_tape, tf.GradientTape() as value_tape:
            probs = self.policy_model(states)
            action_masks = tf.one_hot(actions, self.action_size)
            selected_probs = tf.reduce_sum(probs * action_masks, axis=1)

            # PPO Loss
            loss = -tf.reduce_mean(tf.math.log(selected_probs + 1e-10) * advantages)

            # Value Loss
            value_preds = self.value_model(states)
            value_loss = tf.reduce_mean(tf.square(rewards - value_preds))

        policy_grads = policy_tape.gradient(loss, self.policy_model.trainable_variables)
        value_grads = value_tape.gradient(value_loss, self.value_model.trainable_variables)

        self.policy_optimizer.apply_gradients(zip(policy_grads, self.policy_model.trainable_variables))
        self.value_optimizer.apply_gradients(zip(value_grads, self.value_model.trainable_variables))


In [10]:
env = TwentyQuestionsEnv(keywords)
agent = PPOAgent(state_size=1, action_size=20)

#tf.keras.backend.clear_session()  # Clears the previous model state

for episode in range(10):
    state = env.reset()
    total_reward = 0
    states, actions, rewards = [], [], []

    for t in range(env.max_steps):
        action = agent.get_action(state)
        next_state, reward, done, _ = env.step(action)

        states.append(state)
        actions.append(action)
        rewards.append(reward)

        state = next_state
        total_reward += reward

        if done:
            break

    agent.train(states, actions, rewards)
    print(f"Episode {episode+1}: Total Reward: {total_reward}")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/opt/conda/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-05 07:14:20.820204: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Episode 1: Total Reward: -34.0
Episode 2: Total Reward: -31.0
Episode 3: Total Reward: 11.0
Episode 4: Total Reward: -31.0
Episode 5: Total Reward: -29.5
Episode 6: Total Reward: -32.5
Episode 7: Total Reward: -28.0
Episode 8: Total Reward: -29.5
Episode 9: Total Reward: 18.5
Episode 10: Total Reward: -28.0
